In [1]:
import pandas as pd 
data_df = pd.read_csv('data.csv')
data_df.head()

,question,A,B,C,D,Answer
0,I can't remember _____________ made the teach...,that it was what,what it was that,what was it that,that was it what,B
1,"He let out an ______________ cry, ""we've won!""",excited,exciting,excite,excites,A
2,Is football John's favourite sport? Yes. ____...,Near to,Except,Beside,Next to,D
3,Do you think regular exercise ___________ goo...,benefit from,reach for,make for,go for,C
4,Have you applied ___________ Mr Black _______...,for; to,with; for,with; about,to; for,D


The FitBert model is provided by https://github.com/Qordobacode/fitbert. Some code in this notebook that are used to rank options to fill the blank in the masked string are adapted from the demonstration codes from the url.

In [2]:
from fitbert import FitBert
import numpy as np
from tqdm import tqdm
import re
choice_list = ['A','B','C','D']
fb = FitBert()

device: cuda
using model: bert-large-uncased
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def ans_parse(choice):
    if '/' in choice:
        return ' '
    else:
        choice = choice.strip()
        choice = re.sub(' +',' ',choice)
        return choice.strip()

In [4]:
model_ans_list = []
model_score_list = []
model_choice = ''
# for i in tqdm(range(1)):
for i in tqdm(range(len( data_df ))):
    row = data_df.iloc[i]
    question_original = row['question'].strip()
    
    if ';' in row['A']:
        try:
            if len(row['A'].split(';')) == 2:
                split_list = [[ans_parse(i) for i in row[choice].split(';')] for choice in choice_list]
                masked_string_1 = re.sub('\_+',' ***mask*** ',question_original,1)
                options_1 = list(set( [split[0] for split in split_list] ))
                ranked_options_1 = fb.rank(masked_string_1, options=options_1)
                choice_1 = ' ' if (ranked_options_1[0] == '') else ranked_options_1[0]

                masked_string_2 = fb.fitb(masked_string_1, options=options_1)
                masked_string_2 = re.sub('\_+',' ***mask*** ',masked_string_2,1)
                options_2 = [split[1] for split in split_list if (split[0] == choice_1)]
                ranked_options_2 = fb.rank(masked_string_2, options=options_2)
                choice_2 = ' ' if (ranked_options_2[0] == '') else ranked_options_2[0]
                final_split = [choice_1,choice_2]
                model_choice = choice_list[split_list.index(final_split)]

            elif len(row['A'].split(';')) == 3:
                split_list = [[ans_parse(i) for i in row[choice].split(';')] for choice in choice_list]
                masked_string_1 = re.sub('\_+',' ***mask*** ',question_original,1)
                options_1 = list(set( [split[0] for split in split_list] ))
                ranked_options_1 = fb.rank(masked_string_1, options=options_1)
                choice_1 = ' ' if (ranked_options_1[0] == '') else ranked_options_1[0]

                masked_string_2 = fb.fitb(masked_string_1, options=options_1)
                masked_string_2 = re.sub('\_+',' ***mask*** ',masked_string_2,1)
                options_2 = [split[1] for split in split_list if (split[0] == choice_1)]
                ranked_options_2 = fb.rank(masked_string_2, options=options_2)
                choice_2 = ' ' if (ranked_options_2[0] == '') else ranked_options_2[0]

                masked_string_3 = fb.fitb(masked_string_2, options=options_2)
                masked_string_3 = re.sub('\_+',' ***mask*** ',masked_string_3,1)
                options_3 = [split[2] for split in split_list if ((split[0] == choice_1)&(split[1] == choice_2)) ]
                ranked_options_3 = fb.rank(masked_string_3, options=options_3)
                choice_3 = ' ' if (ranked_options_3[0] == '') else ranked_options_3[0]
                final_split = [choice_1,choice_2,choice_3]
                model_choice = choice_list[split_list.index(final_split)]
        except:
            print(question_original)
            continue

    else:
        masked_string = re.sub('\_+',' ***mask*** ',question_original)
        options = [ans_parse(row['A']), ans_parse(row['B']),ans_parse(row['C']),ans_parse(row['D'])]
        try:
            ranked_options = fb.rank(masked_string, options=options)
        except:
            print(question_original, options)
            continue
        try:
            if ranked_options[0] == '':
                model_choice = choice_list[options.index(' ')]
            else:
                model_choice = choice_list[options.index(ranked_options[0])]
        except:
            print(masked_string,options,ranked_options)


    model_ans_list.append(model_choice)
    if (model_choice == row['Answer'].strip()):
        model_score_list.append(1)
    else: 
        model_score_list.append(0)

(np.array(model_score_list).sum()/len(model_score_list))

100%|██████████| 1183/1183 [13:07<00:00,  1.50it/s]


0.7345731191885038